In [ ]:
import math

import pandas as pd
from neupy import algorithms
import numpy as np
import json
from tqdm import tqdm
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from numpy import sqrt
import math

In [ ]:
def ret(n, n_1):
    return (n_1 - n) / n


PREDICTIONS = 1000
# prices every 15 seconds
# df = pd.read_csv("prices.csv")
f = open("BTC_USDT-3m.json")
df = pd.DataFrame.from_records(json.load(f), columns=['timestamp', 'open', 'high', 'low', 'close', 'vol'])
# df.columns = ['price']
df['ret_c'] = (df['close'] - df['close'].shift(1)) / df['close'].shift(1)
df['ret_o'] = (df['open'] - df['open'].shift(1)) / df['open'].shift(1)

df['ret_h'] = (df['high'] - df['high'].shift(1)) / df['high'].shift(1)
df['ret_l'] = (df['low'] - df['low'].shift(1)) / df['low'].shift(1)
df['ret'] = (df['high'].shift(-1) - df['close']) / df['close']
df['ret_s'] = df['ret'].apply(lambda x:  np.sign(x))
df = df.iloc[0:PREDICTIONS + 3]


def perc_dif(actual, prediction):
    return np.abs(prediction - actual) / prediction

In [ ]:
df

In [ ]:
# OJO CON NDTR=2 y std=0.0029
# OJO CON NDTR=4 y std=0.000075
sq_errors = []
std = 0.0003
hit_rates = []
for NDTR in np.arange(2, 10):
    for std in tqdm(np.concatenate((np.arange(0.001, 1, 0.01), np.arange(0.00001, 0.001, 0.00005), np.arange(1, 10, 0.5)))):
        nans = 0
        actuals = []
        predictions = []
        hits = 0
        for i in np.arange(NDTR, NDTR + PREDICTIONS + 1):
            small_df = df.iloc[i: i + NDTR + 1].copy(deep=True)[['ret_o', 'ret']].reset_index(
                drop=True)
            small_df['ret_o'] = small_df['ret_o'] / small_df['ret_o'].mean()
            small_df['ret'] = small_df['ret'] / small_df['ret'].mean()
            x_train = small_df.iloc[0:NDTR][['ret_o']].values
            y_train = small_df['ret'].iloc[0:NDTR].values
            grnn = algorithms.GRNN(std=std, verbose=False)
            grnn.train(x_train, y_train)
            if len(small_df) > NDTR:
                x_predict = small_df.iloc[NDTR][['ret_o']].values
                prediction = grnn.predict([x_predict])
                if math.isnan(prediction):
                    nans += 1
                    prediction = 0
                else:
                    actual = small_df['ret'].iloc[NDTR]
                    prediction = prediction[0]
                    if np.sign(actual) == np.sign(prediction):
                        hits += 1
                    actuals.append(actual)
                    predictions.append(prediction)
        if len(actuals) > 900:
            hit_rates.append([hits / len(actuals), std, NDTR, len(actuals)])
            # fig = make_subplots()
            # fig.add_trace(
            #     go.Scatter(x=actuals, y=predictions,
            #                name="Retornos", mode='lines+markers'))
            # title= "GRNN NDTR=" + str(NDTR) + "| std=" + str(std) + "|HitRate=" + "{:.2%}".format(hits/len(actuals))
            # fig.update_layout(title=title, xaxis_title="Real", yaxis_title="Prediction")
            # # fig.show()
            # fig.write_image("grnn/prediction_vs_real_NDTR" + str(NDTR) + "_std" + str(std) + "_nosign_nonorm.png")

hit_df = pd.DataFrame(hit_rates, columns=['hit_rate', 'std', 'NDTR', 'predictions'])
hit_df.to_csv("grnn/hits_grnn_nosign_high.csv", header=None, index=None)

hit_df = hit_df[hit_df['predictions'] > 900]
fig = make_subplots()

for NDTR in hit_df['NDTR'].unique():
    actual_hdf = hit_df[hit_df['NDTR'] == NDTR]
    fig.add_trace(
        go.Scatter(x=actual_hdf['std'], y=actual_hdf['hit_rate'].apply(lambda x: 1 - x), name="NDTR=" + str(NDTR),
                   mode="markers+lines"))

fig.update_layout(yaxis_tickformat=".2%", title="Hit rate of prediction for a given std", xaxis_title="STD",
                  yaxis_title="Hit rate", xaxis_tickformat="none")
fig.write_image("grnn/hitrate_nosign_rethigh.png")

fig.show()